In [ ]:
!pip install git+https://github.com/huggingface/peft.git@refs/pull/160/head
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/21955/head
!pip install sentencepiece
!pip install numexpr==2.7.3

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/peft.git (to revision refs/pull/160/head) to /tmp/pip-req-build-dt_xr701
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-dt_xr701
  Running command git fetch -q https://github.com/huggingface/peft.git refs/pull/160/head
  Running command git checkout -q baa2a4d53f4fc337e846808c9feb1253dea081eb
  Resolved https://github.com/huggingface/peft.git to commit baa2a4d53f4fc337e846808c9feb1253dea081eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/21955/head) to /tmp/pip-req-build-eq24sait
  Running command git clone --filter=blob:none --quiet https://github.com/hugging

In [1]:
import argparse
import json
import numpy as np
import random
import tqdm.auto as tqdm

import datasets
import transformers

def proc():
    save_path = "/home/ubuntu/llama_save-30"
    tokenizer_path = "decapoda-research/llama-30b-hf"
    data_path = "/home/ubuntu/train.json"
    max_seq_length = 448 
    
    tokenizer = transformers.LlamaTokenizer.from_pretrained(tokenizer_path)

    all_tokenized = []
    print("reading")
    with open(data_path, "r") as f:
        lines = list(map(json.loads, f.readlines()))
    random.shuffle(lines)
    print("tokenizing")
    for elem in tqdm.tqdm(lines[0:1000]):
        try:
            all_tokenized.append( tokenizer.encode(elem["input"] + elem["output"]) )
        except:
            print("bad line", elem)
            break
            
    all_tokens = [tokenizer.bos_token_id] + [
        tok
        for row in all_tokenized
        for tok in row + [tokenizer.eos_token_id, tokenizer.bos_token_id]
    ]

    truncated_tokens = all_tokens[:(len(all_tokens) //  max_seq_length) *  max_seq_length]
    arr = np.array(truncated_tokens).reshape(-1,max_seq_length)
    ds = datasets.Dataset.from_dict({"input_ids": arr})
    ds.save_to_disk(save_path)
    print(f"Generated {arr.shape[0]} samples.")
    
proc()


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


reading
tokenizing


  0%|          | 0/1000 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/3541 [00:00<?, ? examples/s]

Generated 3541 samples.


In [2]:
import argparse
import os
import math
from dataclasses import dataclass, field
import tqdm.auto as tqdm
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

import datasets
import transformers
from transformers import (
    Trainer,
    TrainingArguments,
)
from peft import (
    get_peft_model,
    LoraConfig,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
    TaskType,
)

def get_peft_config(lora_rank):
       return LoraConfig(
            task_type=TaskType.CAUSAL_LM, 
            inference_mode=False,
            r=lora_rank,
            lora_alpha=32, 
            lora_dropout=0.1,
       )

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        model_out = model(
            input_ids=inputs["input_ids"],
            attention_mask=torch.ones_like(inputs["input_ids"]),
            labels=inputs["input_ids"],  # HF model does the slicing for us
        )
        if return_outputs:
            return model_out.loss, model_out
        else:
            return model_out.loss

def data_collator(features: list) -> dict:
    return {
        "input_ids": torch.stack([
            torch.LongTensor(f["input_ids"])
            for f in features
        ])
    }


def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)

    
def train_llama():
    model_path="decapoda-research/llama-30b-hf"
    dataset_path="/home/ubuntu/llama_save-30/"
    lora_rank = 16
  
    training_args = TrainingArguments(per_device_train_batch_size = 1,
                                        gradient_accumulation_steps = 1,
                                        max_steps = 3500,
                                        learning_rate = 2e-4,
                                        fp16 = True,
                                        logging_steps = 100,
                                        output_dir = "/home/ubuntu/",
                                        optim="adamw_torch",
                                        )

    print("Setup Data")
    dataset = datasets.load_from_disk(dataset_path)

    print("Setup Model")
    model = transformers.LlamaForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16 ,
        #load_in_8bit=True,
        device_map="auto",
    )
    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()
    model.lm_head = CastOutputToFloat(model.lm_head)
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

    print("Setup PEFT")
    peft_config = get_peft_config(lora_rank)
    model = get_peft_model(model, peft_config)
    model = model.cuda()

    print("Train")
    trainer = ModifiedTrainer(
        model=model,
        train_dataset=dataset,
        args=training_args,
        data_collator=data_collator,
    )
    trainer.train()
    save_tunable_parameters(model, os.path.join(training_args.output_dir, "params-30.p"))


In [3]:
train_llama()

Setup Data
Setup Model


Loading checkpoint shards:   0%|          | 0/61 [00:00<?, ?it/s]

Setup PEFT
Train


Step,Training Loss
100,1.702600
200,1.612500
300,1.555100
400,1.563300
500,1.570400
600,1.629600
700,1.680800
800,1.612100
900,1.637700
1000,1.645200


In [1]:
import torch
import transformers
from peft import get_peft_model,LoraConfig
    
def get_peft_config(lora_rank):
       return LoraConfig(
            task_type='lora',
            inference_mode=False,
            r=lora_rank,
            lora_alpha=32, 
            lora_dropout=0.1,
        )
peft_path = "/home/ubuntu/params-30.p"

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = transformers.LlamaForCausalLM.from_pretrained( "decapoda-research/llama-30b-hf")
peft_config = get_peft_config(16)
model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

tokenizer = transformers.LlamaTokenizer.from_pretrained("decapoda-research/llama-30b-hf")







Loading checkpoint shards:   0%|          | 0/61 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [8]:
def f(str):
    inputs = tokenizer(str + "\n\n", return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids=inputs,
                             max_length=500,
                             num_beams=2,
                             repetition_penalty=2.5,
                             length_penalty=0.5,
                             early_stopping=True,
                             no_repeat_ngram_size=2,
                             use_cache=True,
                             do_sample = True,
                             temperature = 1.5,
                             top_k = 50,
                             top_p = 0.95)
    print(tokenizer.decode(outputs[0]))
    


In [22]:
f("Explain why Tom Cruise is so successful")

 Explain why Tom Cruise is so successful

Hollywood A-lister TomCruise celebrates his 60th birthday this month, but he's still going strong with more than 70 movies to his name over a career spanning four decades. His latest film, the blockbuster Top Gun: Maverick, was one of the biggest box office successes of 2022, bringing in $1 billion globally. But what is it that has made him such a hit with audiences? "He doesn't have the classic leading man looks," says BBC entertainment reporter Lizo Mzimba. "But he exudes an extraordinary charm and charisma." As part of our series How To Succeed, we asked three experts to put their finger on what sets Cruisesuccess apart from other stars - and if there are any lessons us ordinary mortals can learn from him. The first person we speak to is Dr Mark O'Connell, reader in cinema studies at Durham University.
*: What makes someone irresistibly charismatic? You might want to watch your friends carefully after finding out who they really are attracte

In [15]:
f("does the white house have a pool")

 does the white house have a pool

The White House has had an indoor swimming pool since 1933, when Franklin D. Roosevelt had one installed and used it to exercise his polio-weakened legs. In 2006, First Lady Laura Bush oversaw the renovation of the pool, which now features a glass wall overlooking the North Lawn. The first outdoor pool at the White Houses West Terrace was built for Chester A. Arthur in ​1881.


In [21]:
f("explain the reasons for the decline of the Ottoman Empire")

 explain the reasons for the decline of the Ottoman Empire

By the late 16th century, it was no longer a threat to Christian Europe. The Ottomans began to lose their control over other parts of their empire as well.
*: They lost some territory in North Africa to rebels who opposed their religious policies and tax system. For example, Algeria became an independent state in the early seventeenth century. Other rebellions broke out in Yemen and Egypt. These conflicts led to power struggles among rulers and increased corruption in government. By the mid-18th century, the provinces had more autonomy than they had had under the earlier sultans. In many cases, local leaders gained more power than the sultan himself.


In [16]:
f("can a prolactinoma be treated with drugs")


 can a prolactinoma be treated with drugs

Prolacatin is a hormone produced by your pituitary gland, which is located at the base of your brain. It's essential for breast development and lactation (milk production). If you have a condition called hyperprolacatinaemia this simply means that your body produces too much of this hiromone - in other words, you are producing more than normal quantities of it on a regular basis. The two most common reasons for having high levels of prolacatin are:
*: taking opioid pain killers or tranquillisers like diazepam (Valium)
1: having a benign (non-cancerous) tumour in your adenohypophysis known as a Prolacaionoma. A prolatcinoma is an overgrowth of one type of cell found in the anterior lobe of the pitutrary Gland.
2: They may occur singly or in multiples within the hypopituitarism 40 percent of cases they are caused by genetic mutations affecting dopamine receptors. You probably won't have any symptoms if you only have slight elevated serum prolaua

In [26]:
f("how do you make cheese")

 how do you make cheese

Cheese production is a very interesting process. As the curd is formed, acidity in the milk is increased by adding rennet and/or lactic acid bacteria. Cheese is produced through the coagulation of milk protein using enzymes called renin or chymosin (rennet). Rennet is traditionally obtained from the stomachs of young mammals, mainly calves, lambs, and goats. These are removed during slaughter for meat production. In modern food production, it is more common to use microorganisms that have been genetically engineered to produce the required proteins as part of their normal activity. Enzyme preparations derived from these microbes are commercially available. The first stage of cheesemaking is known as "preparation," which involves standardizing the fat content of the cow's milk, pasteurization, and heat sterilization. This ensures consistency between batches and eliminates potential pathogens. Pasteurized milk contains around 12-14% butterfat, so if needed, the c

In [27]:
f("list of disney animated movies")


 list of disney animated movies

It's the most wonderful time of the year - and, by that, we mean Disney movie season!
*: The Lion King (1994) *: Pocahontas (2005) This remake of one of Walt Disneys first animated films follows the journey of a young woman named Nani who teams up with her hula dancing pig sidekick, Lilo, to save her island home from an evil villain. It also includes songs written by Phil Collins, as well as the voices of David Ogden Stiers, Tia Carrere, Kevin McDonald, and Zooey Deschanel. While it didn't exactly take off at the box office when it was released back in 26th, Brother Bear has gone on to become a cult classic among Disney fans for its touching story about a brother whose life is turned upside down after he kills a bear cub while out hunting with his older brother. Afterwards, he becomes cursed and transformed into a grizzly bear himself, which leads him on an epic journey to break the curse and find peace within himself. In addition to featuring beautiful

In [28]:
f("classification of hormones")


 classification of hormones

*: Steroid Hormone (e.g., estrogen) - They are synthesized by the cortex of the adrenal gland, placenta, liver, and sex organs; steroids can be water-soluble or fat solubl e.
1: Endocrine Glands 2: Types of Chemical Messengers 3: Functions of Neurotransmitters 4: Synapse 5: Norepinephrine Metabolism 6: Structure of Serotonin 7: Dopamine Biosynthesis 8: Melatonin Receptors 9: Definition of a Cytokine 10: Inflammatory Response


In [29]:
f("genres of jazz music")

 genres of jazz music

Jazz is the most important form of American music in the 20th century. Jazz was born about 1900, mainly in New Orleans and Memphis, Tennessee. It combined elements of European and African-American folk music as well as Ragtime. The first jazz recordings were made by Black players such as Jelly Roll Morton, Sidney Bechet, and Louis Armstrong (1896-1351). In the early days, there were many different kinds of "jazz" groups: trios, quartets, and sextets that played a variety of instruments. These included banjos, guitars, horns, violins, pianos/organs, tubas, basses, drums, etc. Each group had its own special sound and style. Some of the best-known styles are Dixieland, Swing, Bebop, Cool, Fusion, Free Jazz, Avant-garde, Acid, Punk, Smooth, Rap, Latin, Afro-Cuban, Traditional, Third Stream, World Beat, Ska, Techno, Minimalist, Electronic, Ambient, Drum&Bass, Trip-Hop and others. One thing all these styles have in common is improvisation, where musicians make up their

In [30]:
f("explain the weaknesses of the articles of confederation")

 explain the weaknesses of the articles of confederation

The Articles of Confederation had a number of flaws that hindered the United States during the 1780s. A primary flaw was that the central government did not have enough authority to govern properly. Secondly, the Articles lacked checks and balances on the power of each state. Lastly... The Federal Convention met in Philadelphia from May 25 to Sept.
*: Explain how the constitutional convention deviated from its original instructions to draft Amendments to the article of Confredation instead of creating an entirely new goverment. How might the delegates have been able to justify their actions? (4) Constitutional Convention-meet in Philadephia to amend Article of Confratntion but rewrote it


In [31]:
f("explain how to overcome barriers to communication")

 explain how to overcome barriers to communication

Different factors can make communication in the workplace difficult, such as:
*: language and cultural differences. Communication is an important part of any relationship. Being able to get your point across effectively means that people will understand what you have to say, and may be more likely to accept your ideas or requests. If you're not confident about your ability to communicate clearly, then developing this skill can lead to improved relationships with colleagues, managers and clients. Improving your communication skills requires effort and persistence on your part. Some ways you can do this include: Learning to use effective communication strategies; Taking a course in interpersonal communication; Asking for feedback from others about their experiences communicating with you; Practising reflective listening, which involves focusing on what someone else is saying rather than thinking about what they'll say when it's your tur

In [32]:
f("a story about persephone")

 a story about persephone

*: The Greek goddess Demeter, sister of Zeus and mother of Persephones abductor, Hades, was angry that her daughter had been kidnapped and sentenced the world to winter. In an effort to convince his brother to release Peresphone from the land of the dead, Hermes (messenger for the gods) went to King Eleusinian Triptolemus who then traveled to the underworld to plead on behalf of humanity. When he arrived at the gates of Hade's palace, he met Charon, ferryman of souls across the rivers Styx and Acheron. To gain entry into the Underworld, all visitors must first pay him a coin as a fee. Once inside the palace grounds, Cerberus greeted him with three barking heads. As he entered the courtroom, Thanatos - God of Death - sat in judgement next to Elysium Fields. Upon hearing the kings pleas, it was agreed that if Peresephone swore she had eaten nothing while in the Land of Dead, she would be allowed to return to her mother and the Earth would have springtime again.

In [33]:
f("a letter to a sunday keeper")

 a letter to a sunday keeper

It has come to our attention that you are not observing Sunday as a day of rest, but instead worshiping on Sabbath. This is unacceptable for a number of reasons. First and foremost, Gods law states quite clearly in the 10 commandments that we are to keep holy the SABBATH DAY (or Saturday), yet you choose to observe another day with no scriptural support whatsoever. Secondly, if you want to claim that Jesus rose on Sunday, thus making it the true sabbth, then you must also accept that He died on Wednesday which would make Good Friday an incorrect observance. The Bible tells us that there was still darkness over all the land when Jesus gave up the ghost, and at about the ninth hour(3 PM)Jesus cried with a loud voice, saying, Eli,eli, lama sabachthani? That is to say, My God, my God why hast thou forsaken me? Matthew‬ ‭27:45-46‬ So according to this passage, Jesus died just before sunset on Thursday, spent the night time hours between Thurday and Friday being

In [34]:
f("a letter to bank manager")

 a letter to bank manager

*: How to write an application for a loan from the bank, including what you should include in your letter or email and how it should be formatted. If you're thinking about taking out a mortgage with HSBC, read our expert guide to find out more about its different deals, eligibility criteria and customer service. We explain how to apply for one of these loans, as well as who they might be best suited to. An unsecured personal loan allows you to borrow money without having to put up any collateral (such as your home). A good example of when you might need this kind of loan is if you want to take a holiday but dont have the necessary funds. You can then pay back the loan over a set period - usually several years. One of the benefits of applying for this type of credit is that you wont have to worry about losing your house if something happens and you cant keep up repayments. However, lenders are likely to check your credit history before approving your applicati

In [35]:
f("a letter to the hostel warden for leave")

 a letter to the hostel warden for leave

*: Dear Hostel Warden, I am writing to request permission to take leave due to my unavoidable circumstances. Please grant me leave for 10 days starting from (date of leave) to ( date of return). The reason for my leave is that I have been suffering from stomach pain and fever for the past few days. I went to see a doctor at the nearby clinic who asked me to get admitted in the hospital for further treatment. Therefore, as you can understand, it will not be possible for me attend classes or college activities during this time period. Kindly consider my application and let me know about your decision soon. Thanking You Yours Sincerely, Signature


In [36]:
f("an example of checks and balances")

 an example of checks and balances

The Constitution grants the federal government certain enumerated powers, which are spelled out in Article I, Section 8.
*: The power to collect taxes; these must be uniform (the same) throughout the United States. This means that every state pays the same percentage of its income for a particular tax. Congress can determine what type of tax is collected and how it is spent. Taxes have historically been used to pay off national debt, fund infrastructure projects, provide social welfare programs, and more. Federal tax dollars also fund public education, military expenses, space exploration, transportation, scientific research, housing assistance, environmental protections, foreign aid, disaster relief, Social Security, Medicaid, Medicare, unemployment benefits, veteran rehabilitation services, farm subsidies, immigration regulation, health insurance, prison operations, food safety inspection, drug interdiction, law enforcement, border patrol, water pu

In [37]:
f("an algorithm to find the average of three numbers")

 an algorithm to find the average of three numbers

1: If n is a positive integer, then the arithmetic mean of the first n positive integers is given by the formula where k ranges from 1 to n.
2: The middle term of any arithmetic series that has at least three terms can be determined by averaging the two terms that come before and after it.


In [38]:
f("a* algorithm tic tac toe")

 a* algorithm tic tac toe

Today we look at how to use the minimax algorithm to create an unbeatable AI for Tic-Tac-Toe that will never lose a game. This article follows on from our previous articles where we looked at some basic concepts of Artificial Intelligence and then went on to implement a few search algorithms like Breadth First Search, Depth first search and Dijkstra's Algorithm. Today we build upon these concepts and dive into the Minimax Algorithm which is used in games like Chess and Go but can be applied to any 2 player perfect information strategy game such as checkers or even tictactoe. To do this we simply run a loop through all possible permutations of moves until we find one that does not have a move that allows the opponent to make a winning move. If we are unable to find a valid move then it means that there is no way to win so we return None. The problem with greedy algorithms is that they don't take into account all available options when making a decision. In oth

In [39]:
f("the difference between civil rights and civil liberties")

 the difference between civil rights and civil liberties

Civil rights are defined by the U.S. Department of Labor's Employment and Training Administration (ETA) as "all individuals who work for salaries, wages, or other remuneration." Civil rights laws apply to all job applicants and employees on the basis of race, color, religion, sex (including pregnancy), national origin, age, disability, genetic information, citizenship status, and military service.
1: The Fair Labor Standards Act (FLSA) requires employers to comply with minimum wage, overtime pay, child labor, recordkeeping requirements, etc. 3: Title VII of the Civil Rights Act of 1964 prohibits discrimination in hiring, promotion, termination, harassment, compensation, training, fringe benefits, layoffs, retaliation/retaliatory discharge, working conditions, failure to accommodate, a hostile work environment, denial of opportunities, demotion, discipline, unfair scrutiny, adverse action, differential treatment, glass ceilings, 

In [40]:
f("the difference between zucchini and cucumber")

 the difference between zucchini and cucumber

If you've ever taken a stroll through your local grocery store or farmers market, you may have noticed some confusion surrounding these two similar-looking vegetables. While there are slight differences between them, they are often considered interchangeable substitutes in many recipes. However, it's worth knowing exactly what makes each of these unique to help you decide which is right for your next dish. Zucchinis ( Cucurbita pepo ) are dark green squashes that can reach up to 30 inches in length. They belong to the same family as pumpkins, watermelons, gourds, and other types of summer crops. The flesh of a zuchinni has a mild flavor with an almost meaty consistency. This versatile ingredient can be baked, steamed, boiled, grilled, sauteed, or roasted into a variety of delicious side dishes like this Lemon Zoodles recipe. Aside from their culinary uses, raw sliced zukes also make excellent keto diet snacks due to their low carb content.

In [41]:
f("script for a wedding reception")

 script for a wedding reception

Wedding Scripts Sample Wedding Ceremony Script It is an honor and joy to celebrate with you as your family and friends witness the covenant you are making today. As you enter married life, you bring unique personalities, backgrounds, experiences, hopes, and dreams into this relationship. You have much to be thankful for and so many gifts to share with one another. Marriage can sometimes be difficult, but it has the potential to continually draw you closer to each other while strengthening your relationship with God. With all of these factors in mind, I invite you to join hands and face one-another. Please repeat after me: In the sight of God and these witnesses, we ________ and __________ promise to love, honor, cherish, respect, encourage, support, comfort, pray for, trust, forgive, serve, listen to, protect, nurture, sacrifice for… (and add any additional promises you would like to make)…this day and forevermore.


In [42]:
f("what kind of car is lightning mcqueen")

 what kind of car is lightning mcqueen

Car enthusiasts can debate for days on end about which cars are the most iconic in cinematic history. But we'd argue that there's one clear choice at the top of the list: Lightning McQueen from Cars (2006), Piston Cup winner and former Radiator Springs resident. So, as fans gear up for "Cars 3" this weekend, we thought it might be fun to answer the question that has bedeviled them since they first saw the film: What type of race car does he represent?
*: Dale Earnhardt Jr. - No. 8 Chevrolet Monte Carlo SS The legendary driver made his debut in NASCAR'S Busch Series in 1996, piloting a green-and-white Chevy before making the jump to full-time competition in the Cup Series two years later. He was known for both his talent behind the wheel and his popularity among fans, who affectionately referred to him as "Junior. " Although he retired from racing in November after more than two decades with Hendrick Motorsports, he continues to play an active rol

In [43]:
f("what was the great compromise")

 what was the great compromise

*: The Missouri Compromise (1820): Admitted Maine and Missouri into the Union as a free state and a slave state, respectively; extended the 36°30 line westward through the rest of the Louisiana Purchase to the Pacific Ocean. *: Nullification Crisis : John Calhoun supported South Carolina's attempt to nullify the tariffs of 1928 and '32 on the grounds that such taxes were unconstitutional. Although President Andrew Jackson threatened military action if any state attempted to secede from the union, this issue was resolved peacefully by Henry Clay with the Tariff of Abominations in , which lowered the rate of import duties.
o: This crisis showed how fragile the balance of power between North and South really was. o: It also illustrated the need for a more permanent solution to slavery than the Missouri agreement. There were many differences between the north and the south before the civil war but one of them was whether or not new states coming into USA sho

In [49]:
f("what can mimic kidney stone pain")

 what can mimic kidney stone pain

Kidney stones are hard masses of salts and minerals that form in the kidneys when there is a decrease in urine volume and/or an excess of stone-forming substances. There are different types of kidnesy stones. The most common type is made of calcium oxalate. Other types include uric acid, struvite (magnesium, ammonium and phosphate), cystine and silicate.
*: Severe back, side (flank) or abdominal pain; pain radiating to the groin and genitals may also occur.  Pain often starts suddenly and comes in waves lasting 20 to 60 minutes at a time. Sometime pain subsides on its own only to recur days later. Symptoms usually occur when a stone drops into the ureter, causing irritation or blockage. If you have any of these symptoms, make an appointment to see your health care provider. Your provider will review your medical history and ask you about your diet, lifestyle and past health problems. You will be asked if you use medications for blood pressure, cholest

In [45]:
f("how is jello made")

 how is jello made

Jell-O  is a brand name that was created in 1897 by carpenter Pearle Bixby Wait to sell his patented gelatin desserts. It's not the same thing as gelatine or agar, although they are very similar and can be used interchangeably in recipes. Gelatin comes from collagen, one of the main proteins found in connective tissue such as skin, bones, hooves, and hides. The word "gelatin" itself is derived from the Latin word for animal glue. Agar is also a common culinary thickening agent; it comes exclusively from seaweed (usually red algae), rather than an animal source. In addition to being available in powdered form, these ingredients are also sold in sheet form. Both types of sheets are sometimes referred to as "agar strips" or "gelfilm. " They're useful for adding garnish to cakes and other dishes because they don't dissolve until after you cook them. To make your own versions of gelled foods, you need to know a little chemistry. Jellied food relies on hydrocolloids, whic

In [46]:
f("how is osteoporosis diagnosed")

 how is osteoporosis diagnosed

A bone density test can determine if your bones are at risk of breaking. Learn more about getting tested for this silent disease. A DEXA scan is the most common test to measure a persons bonal density, but other tests can be used as well. Find out who should get tested and what options are available. Often called the silent thief, osteoarthritis causes cartilage between joints to break down.
*: What causes it?
1: Who's affected? 2: Symptoms 3: Treatment 4: Prevention


In [47]:
f("how are viruses different from bacteria")

 how are viruses different from bacteria

Bacterial and viral infections have similar symptoms like fever, sore throat, cough, diarrhea, vomiting, fatigue, and aches. However, these two types of infection differ in the following aspects:
1: Duration of Infection Bacterias take longer to replicate than a virus. You can get rid of a bacteraemia through antibiotics in 3-5 days. It takes several weeks to completely recover from an influenza or common cold (caused by rhinoviruses) because they need time to kill the virus inside your body system.
2: Mode of Transmission There are three ways you can be infected with a disease-causing agent: airborne, food/water, or direct contact. Viruses spread mainly through the air when people sneeze or couch around you. They can also spread if you touch contaminated surfaces and then touch your mouth, nose, eyes, etc., without first washing your hands. This mode of transmission is called fecal-oral route. On the other hand, some diseases caused by baceter

In [50]:
f("how is the moon illuminated")

 how is the moon illuminated

The Moon is not a light source, and it does not generate its own light.
*: The Sun lights up the surface of the Moon (the "dayside") so that we can see it in our nighttime sky. *: A Full Moon occurs when the full face of this lit-up side is pointed directly at us. As the Earth revolves around the Sun each month, the same hemisphere faces away from the star. When the half facing towards the sun is visible again, it is called a Waxing Crescent Moon. If you look closely at the image above, you'll notice that the shadows on the Moons surface aren't black. This is because the entire Universe is essentially filled with electromagnetic radiation, most of which is invisible to the human eye. We cannot see infrared or ultraviolet wavelengths, for example, but they do pass through the atmosphere into outer space where they are reflected off the Mooon's surface. Even though the lunar body is about 384,00 kilometers away, some of these photons still make it back to Ea